---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [8]:
import os
os.getcwd()

'/Users/fangyucheng/Documents/code/python_code/daily_code/Social_Network_Analysis'

In [17]:
nx.__version__

'2.2'

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [9]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [10]:
#no companetworkx version

def graph_identification():
    result_list = []
    for graph in P1_Graphs:
        dd = graph.degree()
        dd_list = [value for key, value in dd.items()]
        dd_std = np.std(dd_list)
        if dd_std > 3:
            result_list.append('PA')
        else:
            acc = nx.average_clustering(graph)
            if acc < 0.1:
                result_list.append('SW_H')
            else:
                result_list.append('SW_L')
    return result_list

graph_identification()

AttributeError: 'Graph' object has no attribute '_adj'

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [13]:
G = nx.read_gpickle('email_prediction.txt')
G

In [16]:
G.nodes

AttributeError: 'Graph' object has no attribute '_node'

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [14]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def salary_predictions():
    df = pd.DataFrame(index=G.nodes())
    df['dc'] = pd.Series(nx.degree_centrality(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['cc'] = pd.Series(nx.closeness_centrality(G))
    df['bc'] = pd.Series(nx.betweenness_centrality(G))

    node_dict = G.node
    result_list = []
    for key, value in node_dict.items():
        new_dict = {'node': key,
                   'managementsalary': value['ManagementSalary']}
        result_list.append(new_dict)

    result_df = pd.DataFrame(result_list)

    merged_df = df.merge(result_df, how='inner', left_index=True, right_index=True)
    test_df = merged_df[np.isnan(merged_df['managementsalary'])]
    train_df = merged_df[~np.isnan(merged_df['managementsalary'])]
    feature = ['dc', 'clustering', 'cc', 'bc']
    y_train = train_df['managementsalary']
    X_train = train_df[feature]
    X_test = test_df[feature]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, y_train)
    test_proba = clf.predict_proba(X_test_scaled)
    pro = test_proba[:,1]  
    return pd.Series(pro, X_test.index)    

salary_predictions()

AttributeError: 'Graph' object has no attribute '_node'

In [58]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


488446

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    
    #preferetial attachment model
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    
    #cn_soundarajan_hopcroft
    cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))
    df_cn_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df_cn_soundarajan_hopcroft['cn_soundarajan_hopcroft'] = [x[2] for x in cn_soundarajan_hopcroft]
    df = df.join(df_cn_soundarajan_hopcroft,how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    
    #resource allocation index
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    
    #jaccard coefficient
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df = future_connections.join(df,how='outer')
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    predictions = pd.Series(test_proba, X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[x] for x in target.index]
    return target['prob']

new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.030926
(542, 751)    0.012658
(20, 426)     0.587821
(50, 989)     0.012756
(942, 986)    0.012862
(324, 857)    0.012742
(13, 710)     0.150667
(19, 271)     0.117999
(319, 878)    0.012790
(659, 707)    0.012668
(49, 843)     0.012821
(208, 893)    0.012691
(377, 469)    0.009046
(405, 999)    0.020911
(129, 740)    0.019202
(292, 618)    0.020924
(239, 689)    0.012797
(359, 373)    0.010447
(53, 523)     0.079130
(276, 984)    0.012815
(202, 997)    0.012836
(604, 619)    0.047584
(270, 911)    0.012795
(261, 481)    0.065936
(200, 450)    0.998744
(213, 634)    0.012627
(644, 735)    0.035625
(346, 553)    0.012384
(521, 738)    0.011920
(422, 953)    0.019322
                ...   
(672, 848)    0.012795
(28, 127)     0.979580
(202, 661)    0.012343
(54, 195)     0.999994
(295, 864)    0.012727
(814, 936)    0.012642
(839, 874)    0.012862
(139, 843)    0.012735
(461, 544)    0.011600
(68, 487)     0.011545
(622, 932)    0.012703
(504, 936)    0.017945
(479, 528) 